# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: __Jhoany Rodriguez, Andre Paredes y Fabian Galarza__
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [3]:
import pandas as pd
df_neoauto = pd.read_csv('neoauto_20240924.csv')
mean_value = df_neoauto.groupby(['item_brand', 'item_category_2'], as_index=False)['value'].mean()
print(mean_value)

       item_brand item_category_2          value
0      ALFA-ROMEO       Hatchback    8500.000000
1      ALFA-ROMEO           Sedan    2500.000000
2    ASTON MARTIN  Camionetas Suv       0.000000
3    ASTON MARTIN       Deportivo       0.000000
4    ASTON MARTIN           Sedan  149900.000000
..            ...             ...            ...
197         VOLVO  Camionetas Suv   41021.219512
198         VOLVO       Deportivo    8800.000000
199         VOLVO       Hatchback   12272.500000
200         VOLVO           Sedan   10802.777778
201         VOLVO   Station Wagon    6100.000000

[202 rows x 3 columns]


2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [5]:
precio_promedio_general = df_neoauto['item_price'].mean()
autos_filtro = df_neoauto[(df_neoauto['item_km'] > 100000) & (df_neoauto['item_price'] < precio_promedio_general)]
cantidad_autos = autos_filtro.shape[0]
print(cantidad_autos)

497


3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [6]:
modelo_mas_caro_barato = df_neoauto.groupby('item_brand').agg(
    modelo_mas_caro=('item_name', lambda x: x[df_neoauto.loc[x.index, 'item_price'].idxmax()]),
    precio_mas_caro=('item_price', 'max'),
    modelo_mas_barato=('item_name', lambda x: x[df_neoauto.loc[x.index, 'item_price'].idxmin()]),
    precio_mas_barato=('item_price', 'min'))
print(modelo_mas_caro_barato)

                       modelo_mas_caro  precio_mas_caro  \
item_brand                                                
ALFA-ROMEO         alfa-romeo 147 2006           8500.0   
ASTON MARTIN     aston martin dbx 2021         229000.0   
AUDI                    audi rs 6 2024         220000.0   
AUSTIN         austin mini cooper 1970              1.0   
BAIC                     baic x55 2021          12800.0   
...                                ...              ...   
SWM                   swm g05 pro 2022          14000.0   
TOYOTA               toyota hilux 2019         100000.0   
TRIUMPH              triumph tr-7 1971          18900.0   
VOLKSWAGEN    volkswagen teramont 2023          48900.0   
VOLVO                 volvo xc-90 2023          84990.0   

                           modelo_mas_barato  precio_mas_barato  
item_brand                                                       
ALFA-ROMEO           alfa-romeo alfasud 1982             2500.0  
ASTON MARTIN          aston martin

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [9]:
marcas_top_financiadas = df_neoauto[df_neoauto['item_financed_by'] == 'Santander'].groupby('item_brand')['item_brand'].count().nlargest(3)
print("Las tres marcas con mayor número de autos financiados por Santander son:")
print(marcas_top_financiadas)


Las tres marcas con mayor número de autos financiados por Santander son:
item_brand
BMW       296
TOYOTA    261
NISSAN    228
Name: item_brand, dtype: int64


5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [14]:
desvest_precio_transmision = df_neoauto.groupby('item_transmission')['item_price'].std()
print("Desviación estándar del precio por tipo de transmisión:")
print(desvest_precio_transmision)


Desviación estándar del precio por tipo de transmisión:
item_transmission
Automática                 15332.729048
Automática - Secuencial    22678.641589
Mecánica                    8137.122127
Name: item_price, dtype: float64


6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [15]:
kilometraje_tags = df_neoauto[df_neoauto['item_tag'].isin(['Premium', 'Como nuevo'])].groupby('item_tag')['item_km'].mean()
print("Kilometraje promedio por tag:")
print(kilometraje_tags)


Kilometraje promedio por tag:
item_tag
Como nuevo    35963.476190
Premium       48400.235897
Name: item_km, dtype: float64


7. ¿Cuáles son las marcas de vehículos para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [37]:
from datetime import datetime

publicaciones_validas = df_neoauto[df_neoauto['item_price'].notna()]

año_actual = 2024 # Asumimos que el año actual es 2024
año_limite = año_actual - 5

autos_recientes = publicaciones_validas[publicaciones_validas['item_year'] >= año_limite]

publicaciones_por_modelo = autos_recientes.groupby(['item_brand', 'item_name']).size().reset_index(name='count') # publicaciones únicas por marca y modelo

modelos_con_mas_de_5_publicaciones = publicaciones_por_modelo[publicaciones_por_modelo['count'] >= 5]

modelos_por_marca = modelos_con_mas_de_5_publicaciones.groupby('item_brand')['item_name'].nunique().reset_index(name='num_modelos') # se agrupa por marca y se cuenta cuántos modelos cumplen el criterio

modelos_totales_recientes = autos_recientes.groupby('item_brand')['item_name'].nunique().reset_index(name='total_modelos') # se cuentan todos los modelos recientes (últimos 5 años) por marca

marcas_validas = pd.merge(modelos_por_marca, modelos_totales_recientes, on='item_brand')

marcas_finales = marcas_validas[marcas_validas['num_modelos'] == marcas_validas['total_modelos']]['item_brand'] # filtro: TODOS los modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una

print("Marcas con todos sus modelos recientes (últimos 5 años) con al menos 5 publicaciones válidas:")
print(marcas_finales.tolist()) # no existen marcas que cumplan con el requisito

Marcas con todos sus modelos recientes (últimos 5 años) con al menos 5 publicaciones válidas:
[]


## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [ ]:
import pandas as pd

# Ruta al archivo original
file_path = "C:/Users/HP/Downloads/pwt1001.xlsx"

# Cargar la hoja "Data"
data = pd.read_excel(file_path, sheet_name="Data")

# 1. 
data['rgdpe_pc'] = data['rgdpe'] / data['pop']  # PBI per cápita

# Filtrar los datos válidos
valid_data = data.dropna(subset=['rgdpe_pc', 'year'])
latest_year = valid_data['year'].max()
top_countries = valid_data[valid_data['year'] == latest_year].nlargest(10, 'rgdpe_pc')[['country', 'rgdpe_pc']]

print("Top países por PBI per cápita PPP ajustado:")
print(top_countries)

2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [ ]:
# 2.
data_1990_2019 = valid_data[(valid_data['year'] >= 1990) & (valid_data['year'] <= 2019)]
df_changes = data_1990_2019.pivot(index='country', columns='year', values='rgdpe_pc')
df_changes['change'] = df_changes[2019] - df_changes[1990]
ranking_changes = df_changes.nlargest(10, 'change')[['change']]
print("Ranking de países con mayor incremento (1990-2019):")
print(ranking)
print("Ubicación de Perú:", ranking_peru)

3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [ ]:
# 3. 
data_2010_2019 = valid_data[(valid_data['year'] >= 2010) & (valid_data['year'] <= 2019)]
human_capital_avg = data_2010_2019.groupby('country')['hc'].mean().reset_index()
relation_hc_gdp = data_2010_2019.groupby('country').agg({
    'rgdpe_pc': 'mean',
    'hc': 'mean',
    'avh': 'mean'
}).reset_index()
print("Promedio del índice de capital humano (2010-2019):")
print(human_capital_avg)

4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [ ]:
# 4. 

# Calcular inflación
data['inflation'] = data.groupby('country')['pl_con'].pct_change() * 100

# Identificar los países y años con mayor inflación
max_inflation = data.loc[data.groupby('country')['inflation'].idxmax()][['country', 'year', 'inflation']]

# Mostrar resultados de la mayor inflación anual
print("País con mayor inflación anual registrada:")
print(max_inflation)

5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [ ]:
# Solución

6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [ ]:
# Solución

7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [ ]:
# Solución

## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [ ]:
# Solución

9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [ ]:
# Solución

10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [ ]:
# Solución